# CIFAR-10 Classification
> Performance analysis for CIFAR-10 Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [CIFAR-10,Rooflines,Performance Prediction]
- image: images/cifar_logo.png

In [1]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

csv_path = "./data/cleaned_csv/backup.csv"

# Theretical Analysis of CIFAR-10

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input

#first process the following csv's to get clean ready-to-plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(dataframe = pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 
          neural_network = 'cifar')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for CIFAR-10 classification. The metric used for the theoretical performance is input/second.
We observe that prunning along with quantization outputs some of the best performance results.

In [3]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load cifar10 dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_cifar10.csv"), 'pink', 'Performance prediction for CIFAR-10')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for CIFAR-10

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [4]:
#hide
%run scripts/altair_plots.py   #get table with the experiments overview
print(pd.read_csv('data/overview_experiments_cifar10.csv').to_markdown())

|    | Hardware   | Platform         | CNV                              | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:---------------------------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | na                               | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | na                               | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | na                               | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | [INT2,INT4]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | [INT2,INT4]*[100%,50%,25%,12.5%] | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | [FP16,FP32]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]               |
|  6 | GPU        | TX2-maxp         | [FP16,FP32]*[100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]

In [5]:
#hide_input
%run scripts/altair_plots.py  #get table with the experiments overview
tableOverviewExperiments(['data/overview_experiments_cifar10_.csv'])

In [6]:
#hide
#%writefile scripts/utils.py
def norm_by_group(df, column, group_col):
    """ Normalizes pandas series by group """
    df["norm-"+column] = df.groupby(group_col)[column].apply(lambda x: (x / x.max()))
    return df

def select_color(sel, column):
    """ Easy way to set colors based on selection for altair plots
    """
    return alt.condition(sel, 
                      alt.Color(column),
                      alt.value('lightgray'))

def get_pareto_df(df, groupcol, xcol, ycol):
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

def label_point(x, y, val, ax, rot=0):
    """ from https://stackoverflow.com/questions/46027653/adding-labels-in-x-y-scatter-plot-with-seaborn"""
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']), rotation=rot)

In [7]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "GPU" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
maxp_df["hw_quant_prun"] = maxp_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
cnv_df = maxp_df[(maxp_df["NN_Topology"] == "CNV") & maxp_df['lat-comp'].notna() & maxp_df["top1 [%]"].notna()]
cnv_df["hw_quant_prun"] = cnv_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
cnv_df["PruningFactor"] = cnv_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(cnv_df, "lat-comp", "NN_Topology");
cnv_df["quant_model"] = cnv_df.Precision + '_' + cnv_df.HWType
cnv_df.rename(columns={"top1 [%]": "top1"}, inplace=True)

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [8]:
#hide
figa_df = cnv_df[(cnv_df["HWType"].isin(["NCS", "ZCU104-Bismo", "U96-Quadcore A53"]))]
figb_df = cnv_df[(cnv_df["HWType"].isin(["GPU", "ZCU104-FINN", "U96-Quadcore A53"]))]

### Line Plot

In [9]:
#hide_input
fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_quant_prun"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency versus Performance for Pruned and Quantized CNV Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

alt.HConcatChart(...)

### Boxplots

In [10]:
#hide_input
box1 = alt.Chart(cnv_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("lat-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Latency by Hardware/Framework and Pruning for CNV"
)
box1.interactive()

alt.FacetChart(...)

In [11]:
#hide_input
box1 = alt.Chart(cnv_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("fps-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Throughput by Hardware/Framework and Pruning for CNV",
).interactive()
box1

alt.FacetChart(...)

In [12]:
#hide_input
box1 = alt.Chart(cnv_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("Full_Pwr_W", scale=alt.Scale( type="log")),
    color=alt.Color('PruningFactor:O', legend=alt.Legend(columns=1, title = "Pruning Factor")),
).facet(column="quant_model").properties(
    title="Power Consumption by Hardware/Framework and Pruning for CNV",
).interactive()
box1

alt.FacetChart(...)

### Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [13]:
#hide_input
cnv_pareto = get_pareto_df(cnv_df, 'hw_quant_prun', 'fps-comp', 'top1')

cnv_lines = alt.Chart(cnv_df).mark_line(point=True).encode(
    x="fps-comp",
    y=alt.Y("top1:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("hw_quant_prun", legend=alt.Legend(columns=1, title = "Hardware_Quantization_Pruning")),
    tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", "top1", "fps-comp"],
)
cnv_pareto_plot = alt.Chart(cnv_pareto).mark_line().encode(
    x="x",
    y=alt.Y("y", scale=alt.Scale(zero=False)),
)
(cnv_lines+cnv_pareto_plot).interactive().properties(
    width=W,
    height=H,
    title="cnv Cassification Design Space: Accuracy versus Performance"
)

alt.LayerChart(...)

In [14]:
#hide
cnv_df.to_csv('data/cleaned_csv/experimental_data_cifar.csv', index = False)
cnv_df.head()

,NN_Topology,HWType,Precision,Op mode,batch/thread/stream,lat-comp,fps-system,fps-comp,tp-system,tp-comp,...,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_quant_prun,norm-lat-comp,quant_model
0,CNV,NCS,FP16,na,1,4.97914,151.555,200.838,NaN,NaN,...,NaN,0.53,1.2,1.728796,0.469450,100.0,l3,NCS_FP16_100%,0.000342,FP16_NCS
1,CNV,NCS,FP16,na,2,8.93829,162.738,223.757,NaN,NaN,...,NaN,0.53,1.2,1.780300,0.938899,100.0,l3,NCS_FP16_100%,0.000614,FP16_NCS
2,CNV,NCS,FP16,na,4,16.84730,171.284,237.426,NaN,NaN,...,NaN,0.53,1.2,1.823024,1.877799,100.0,l3,NCS_FP16_100%,0.001157,FP16_NCS
3,CNV,NCS,FP16,na,8,31.86580,177.259,251.053,NaN,NaN,...,NaN,0.53,1.2,1.860921,3.755598,100.0,l3,NCS_FP16_100%,0.002188,FP16_NCS
4,CNV,NCS,FP16,na,16,61.77320,181.621,259.012,NaN,NaN,...,NaN,0.53,1.2,1.879459,7.511196,100.0,l3,NCS_FP16_100%,0.004242,FP16_NCS
